# **Try! SimpleTransformer Approach**

In [ ]:
!pip install simpletransformers

# **Import Necessary Library**

In [ ]:
import os,re ,string
import random
import numpy as np
import pandas as pd
import sklearn

import torch

from simpletransformers.classification import ClassificationModel, ClassificationArgs
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

# **Seed**

In [ ]:
seed = 1337

random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.backends.cudnn.deterministic = True

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname,filename))

# **Load Input data**

In [ ]:
train = pd.read_csv("../input/us-patent-phrase-to-phrase-matching/train.csv")
test = pd.read_csv("../input/us-patent-phrase-to-phrase-matching/test.csv")
sample = pd.read_csv("../input/us-patent-phrase-to-phrase-matching/sample_submission.csv")
print(f'train_shape: {train.shape},test_shape: {test.shape},sample_shape: {sample.shape}')
train.head()


In [ ]:
train.columns

# **Feature selection**

In [ ]:
train = train[['context','score']]

# **Default model - BERT**

In [ ]:
#default model
bert_uncased = ClassificationModel('bert','bert-large-uncased')

#print all argument of model
bert_uncased.args

In [ ]:
# tweak based on default argument above

custom_args = {'fp16': False, #not using mixed precision
              'train_batch_size': 4, #default is 8
              'gradient_accumulations_steps':2,
              'do_lower_case': True,
              'learning_rate': 1e-05, # using lower lr rate
              'overwrite_output_dir': True, #important for cv
              'num_train_epochs': 2 }# default is 1

# **KFOLD - 5**

In [ ]:
#fold

n = 5
kf = KFold(n_splits = n)
results = []

for train_index, val_index in kf.split(train):
    train_df = train.iloc[train_index]
    val_df = train.iloc[val_index]
    
    model = ClassificationModel('bert', 'bert-base-uncased',args=custom_args)
    model.train_model(train_df)
    
    result,model_outputs, wrong_predictions = model.eval_model(val_df,acc=sklearn.metrics.accuracy_score)
    
    print(result['acc'])
    results.append(result['acc'])

In [ ]:
for i,result in enumerate(results,1):
    print(f"Fold-{i}:{result}")
    
print(f"{n} - fold CV accuracy result: Mean: {np.mean(results)} Standard deviation: {np.std(results)}")

# **Full Training**

In [ ]:
model = ClassificationModel('bert','bert-base-uncased',args = custom_args)
model.train_model(train)

# **Prediction**

In [ ]:
test= pd.read_csv('../input/us-patent-phrase-to-phrase-matching/test.csv')
predictions, raw_outputs = model.predict(test['context'])

sample_submission = pd.read_csv("../input/us-patent-phrase-to-phrase-matching/sample_submission.csv")
sample_submission["score"] = predictions
sample_submission.to_csv("submission.csv", index=False)


# **This person introduce new one Simple transformer @szelee**

## **Full credit : https://www.kaggle.com/code/szelee/simpletransformers-hyperparam-tuning-k-fold-cv**